# Predicting customer spend

Thomas Wood

fastdatascience.com

Data science consulting

The dataset I used is from Kaggle, https://www.kaggle.com/c/customer-spend-model

The YouTube video tutorial ~is at https://studio.youtube.com/channel/UCLPrDH7SoRT55F6i50xMg5g

In [2]:
%matplotlib inline
from sqlalchemy import create_engine
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

sqlEngine = create_engine('mysql+pymysql://thomas:@127.0.0.1', pool_recycle=3600)
dbConnection = sqlEngine.connect()
dbConnection.execute("use purchases;")

In [3]:
pd.read_sql_query("SELECT id, orddate, ordnum, category, price FROM orders LIMIT 10", dbConnection)

,id,orddate,ordnum,category,price
0,957,2008-02-10,38650,35,5.010658
1,957,2008-02-10,38650,35,20.426102
2,957,2008-02-10,38650,19,20.400543
3,957,2008-03-15,48972,40,25.539017
4,957,2008-11-22,150011,40,14.316170
5,957,2008-11-22,150011,40,8.589699
6,957,2009-10-03,286151,19,15.313187
7,957,2010-04-04,376779,14,12.782295
8,957,2010-04-04,376779,14,5.087353
9,957,2010-04-04,376779,35,6.544533


In [4]:
dbConnection.execute("""CREATE OR REPLACE VIEW customerDailySpend AS SELECT id, 
       orddate, 
       Sum(price) AS dailySpend
       FROM orders
       GROUP BY id, orddate
       ORDER BY id, orddate;
""")

In [5]:
pd.read_sql_query("""SELECT * FROM customerDailySpend;""", dbConnection)

,id,orddate,dailySpend
0,957,2008-02-10,45.837303
1,957,2008-03-15,25.539017
2,957,2008-11-22,22.905869
3,957,2009-10-03,15.313187
4,957,2010-04-04,24.414181
5,957,2011-08-14,34.103157
6,957,2011-09-10,35.795532
7,957,2011-10-10,45.786179
8,957,2012-06-19,27.900585
9,957,2012-07-28,12.949997


In [6]:
df_features_and_spends = pd.read_sql_query("""SELECT id, 
       orddate, 
       DAYOFWEEK(orddate) as weekday,
       Sum(dailySpend) 
         OVER ( 
           partition BY id 
           ORDER BY id, orddate rows BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) as TotalPrecedingSpend,
    Avg(dailySpend) 
         OVER ( 
           partition BY id 
           ORDER BY id, orddate rows BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) as AvgPrecedingSpend,
    Sum(dailySpend) 
         OVER ( 
           partition BY id 
           ORDER BY id, orddate rows BETWEEN 1 PRECEDING AND 1 PRECEDING) as PreviousSpend,
    Count(dailySpend) 
         OVER ( 
           partition BY id 
           ORDER BY id, orddate rows BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) as NumPrecedingVisits,
    dailySpend
FROM   customerDailySpend 
GROUP  BY id, orddate
ORDER BY id, orddate """, dbConnection)
df_features_and_spends

,id,orddate,weekday,TotalPrecedingSpend,AvgPrecedingSpend,PreviousSpend,NumPrecedingVisits,dailySpend
0,957,2008-02-10,1,NaN,NaN,NaN,0,45.837303
1,957,2008-03-15,7,45.837303,45.837303,45.837303,1,25.539017
2,957,2008-11-22,7,71.376320,35.688160,25.539017,2,22.905869
3,957,2009-10-03,7,94.282188,31.427396,22.905869,3,15.313187
4,957,2010-04-04,1,109.595375,27.398844,15.313187,4,24.414181
5,957,2011-08-14,1,134.009556,26.801911,24.414181,5,34.103157
6,957,2011-09-10,7,168.112713,28.018785,34.103157,6,35.795532
7,957,2011-10-10,2,203.908245,29.129749,35.795532,7,45.786179
8,957,2012-06-19,3,249.694424,31.211803,45.786179,8,27.900585
9,957,2012-07-28,7,277.595009,30.843890,27.900585,9,12.949997


In [7]:
df_features_and_spends.dropna(inplace=True)

In [ ]:
sns.barplot(x="weekday", y="dailySpend", data=df_features_and_spends)

C:\Users\woodt.DESKTOP-QB0N6BQ\Anaconda3\lib\site-packages\scipy\stats\stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


# Start machine learning modelling

In [ ]:
m = RandomForestRegressor()

In [ ]:
plt.hist(df_features_and_spends.orddate);

In [ ]:
df_train = df_features_and_spends[df_features_and_spends.orddate < pd.Timestamp(2014,1,1)]
df_test = df_features_and_spends[df_features_and_spends.orddate >= pd.Timestamp(2014,1,1)]

In [ ]:
m.fit(df_train.drop(["id", "orddate", "dailySpend"], axis=1), df_train.dailySpend)

In [ ]:
y_pred = m.predict(df_test.drop(["id", "orddate", "dailySpend"], axis=1))

In [ ]:
y_pred

In [ ]:
df_test["y_pred"] = y_pred

In [ ]:
df_test

In [ ]:
residuals = df_test.y_pred - df_test.dailySpend

In [ ]:
plt.hist(residuals, range=(-50,50));

In [ ]:
mean_absolute_error(df_test.dailySpend, df_test.y_pred)

# Prediction for the future

In [ ]:
df_features_and_spends_for_prediction = pd.read_sql_query("""SELECT id,
    2 as weekday,
       Sum(dailySpend)  as TotalPrecedingSpend,
    Avg(dailySpend) 
         as AvgPrecedingSpend,
        LAST_VALUE(dailySpend) OVER (partition by id order by id, orddate) as PreviousSpend,
    Count(dailySpend)  as NumPrecedingVisits
FROM   customerDailySpend 
GROUP  BY id
ORDER BY id """, dbConnection)
df_features_and_spends_for_prediction

In [ ]:
y_pred_future = m.predict(df_features_and_spends_for_prediction.drop(["id"], axis=1))

In [ ]:
df_features_and_spends_for_prediction["y_pred_future"] = y_pred_future

In [ ]:
df_features_and_spends_for_prediction